In [17]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [18]:
train_dir = 'data/'

In [19]:
# image data generator - data augmentation
train_gen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range = 20,
    zoom_range=0.2,
    horizontal_flip=True)


# create training object
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,           #no of images per batch
    class_mode='binary',
    subset='training')

# create validation object
val_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=32,           #no of images per batch
    class_mode='binary',
    subset='validation')

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [20]:
# build model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential

model = Sequential([
    Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)),
    MaxPooling2D(2,2),

    Conv2D(64,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),

    Conv2D(128,(3,3), activation = 'relu'),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation= 'sigmoid')
])

In [21]:
# compile model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
# train model
history = model.fit(train_data, validation_data = val_data, epochs = 5)

Epoch 1/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 121s 630ms/step - accuracy: 0.8282 - loss: 0.3945 - val_accuracy: 0.9000 - val_loss: 0.2523
Epoch 2/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 145s 766ms/step - accuracy: 0.8885 - loss: 0.2649 - val_accuracy: 0.9391 - val_loss: 0.1757
Epoch 3/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.9173 - loss: 0.2166 - val_accuracy: 0.9391 - val_loss: 0.1683
Epoch 4/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 183s 969ms/step - accuracy: 0.9221 - loss: 0.1991 - val_accuracy: 0.9391 - val_loss: 0.1778
Epoch 5/5
189/189 ━━━━━━━━━━━━━━━━━━━━ 170s 897ms/step - accuracy: 0.9317 - loss: 0.1869 - val_accuracy: 0.9417 - val_loss: 0.1530


In [23]:
# for my input
from tensorflow.keras.preprocessing import image

img_path = 'sample.jpg'
img = image.load_img(img_path, target_size = (150,150))
img_array = image.img_to_array(img)/255.0
img_array = np.expand_dims(img_array, axis = 0)

prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("Prediction: Without mask")
else:
    print("Prediction: With mask")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
Prediction: With mask


In [25]:
model.save("FaceMaskDetection.keras")